**Function 4:**

Fast, but Inaccurate Modelling. This example is for a particular business relying heavily on online sales. It can run very accurate calculations to figure out what is the optimal placement of their product across warehouses. Unfortunately, the calculations are extremely expensive (computationally) to run, so they can only do it once every two weeks. Instead, they propose using a machine learning model which approximates the solution quickly (in a few hours). The model has four hyper-parameters you need to tune, and the output corresponds to the difference between the expensive calculation, and the model. Since you are modelling a dynamical system, expect a lot of local optima!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
#from statistics import mean, median, mode, stdev,NormalDist

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
#from scipy.stats import norm

from statistics import NormalDist
import itertools as it

In [2]:
f4_inputs = np.load("initial_inputs.npy")
f4_outputs = np.load("initial_outputs.npy")
f4_inputs_2 = np.load("initial_inputs2.npy")
f4_outputs_2 = np.load("initial_outputs2.npy")

In [3]:
f4_inputs = np.concatenate((f4_inputs, f4_inputs_2)) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.533179, 0.320368, 0.682501, 0.312599]])))
f4_inputs = np.concatenate((f4_inputs, np.array([[0.444629, 0.320368, 0.808875, 0.908167]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.607765, 0.358771, 0.405825, 0.251007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.687501, 0.84587 , 0.833788, 0.143186]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.557765, 0.448771, 0.445825, 0.269007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.507765, 0.498771, 0.395825, 0.319007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.457765, 0.548771, 0.345825, 0.369007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.407765, 0.498771, 0.295825, 0.419007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.357765, 0.448771, 0.245825, 0.469007]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.325901, 0.423101, 0.406001, 0.415501]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.287012, 0.439767, 0.456001, 0.432167]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.38256 , 0.363705, 0.389898, 0.390221]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.331802, 0.389754, 0.375876, 0.369452]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.331882, 0.379388, 0.371487, 0.422793]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.301994, 0.331229, 0.44261 , 0.408753]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.402732, 0.392983, 0.412051, 0.411811]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.394307, 0.324568, 0.266869, 0.332541]]))) 
f4_inputs = np.concatenate((f4_inputs, np.array([[0.395203, 0.398125, 0.414027, 0.449168]]))) 

f4_outputs = np.concatenate((f4_outputs,f4_outputs_2))
f4_outputs = np.append(f4_outputs,-7.457831737943366)
f4_outputs = np.append(f4_outputs,-20.190789652235313)
f4_outputs = np.append(f4_outputs,-4.588426873216477)
f4_outputs = np.append(f4_outputs,-24.332647737297297)
f4_outputs = np.append(f4_outputs,-3.685845588546609)
f4_outputs = np.append(f4_outputs,-2.8122115680390354)
f4_outputs = np.append(f4_outputs,-2.590293835023562)
f4_outputs = np.append(f4_outputs,-2.0998084663837484)
f4_outputs = np.append(f4_outputs,-2.368538951646705)
f4_outputs = np.append(f4_outputs,0.13490612991991258)
f4_outputs = np.append(f4_outputs,-1.3532237087009196)
f4_outputs = np.append(f4_outputs,0.15448865125965527)
f4_outputs = np.append(f4_outputs,-0.02620181983396419)
f4_outputs = np.append(f4_outputs,0.19106899681514777)
f4_outputs = np.append(f4_outputs,-1.0065878357155316)
f4_outputs = np.append(f4_outputs,0.48394520320252754)
f4_outputs = np.append(f4_outputs,-1.70575402025506)
f4_outputs = np.append(f4_outputs,-0.1020328389196119)


In [4]:
f4_inputs

array([[0.89698105, 0.72562797, 0.17540431, 0.70169437],
       [0.8893564 , 0.49958786, 0.53926886, 0.50878344],
       [0.25094624, 0.03369313, 0.14538002, 0.49493242],
       [0.34696206, 0.0062504 , 0.76056361, 0.61302356],
       [0.12487118, 0.12977019, 0.38440048, 0.2870761 ],
       [0.80130271, 0.50023109, 0.70664456, 0.19510284],
       [0.24770826, 0.06044543, 0.04218635, 0.44132425],
       [0.74670224, 0.7570915 , 0.36935306, 0.20656628],
       [0.40066503, 0.07257425, 0.88676825, 0.24384229],
       [0.6260706 , 0.58675126, 0.43880578, 0.77885769],
       [0.95713529, 0.59764438, 0.76611385, 0.77620991],
       [0.73281243, 0.14524998, 0.47681272, 0.13336573],
       [0.65511548, 0.07239183, 0.68715175, 0.08151656],
       [0.21973443, 0.83203134, 0.48286416, 0.08256923],
       [0.48859419, 0.2119651 , 0.93917791, 0.37619173],
       [0.16713049, 0.87655456, 0.21723954, 0.95980098],
       [0.21691119, 0.16608583, 0.24137226, 0.77006248],
       [0.38748784, 0.80453226,

In [5]:
f4_outputs

array([-2.21082878e+01, -1.46013966e+01, -1.16999325e+01, -1.60537651e+01,
       -1.00696334e+01, -1.54870825e+01, -1.26816850e+01, -1.60263998e+01,
       -1.70492346e+01, -1.27417660e+01, -2.73163964e+01, -1.35276489e+01,
       -1.66791152e+01, -1.65071586e+01, -1.78179993e+01, -2.65618208e+01,
       -1.27583242e+01, -1.94415576e+01, -2.89032737e+01, -1.37027469e+01,
       -2.94270914e+01, -1.15657420e+01, -2.68577864e+01, -7.96677535e+00,
       -6.70208925e+00, -3.26256602e+01, -1.99894979e+01, -4.02554228e+00,
       -1.31227823e+01, -2.31394284e+01, -1.11695198e+01, -1.63244903e+01,
       -3.13844591e+01, -2.72948006e+01, -2.38556543e+01, -1.41568796e+01,
       -2.67497513e+01, -1.51999203e+01, -9.16936028e+00, -1.34622659e+01,
       -6.41942649e+00, -1.68006221e+01, -4.27992993e+00, -1.62096673e+01,
       -2.46387788e+01, -7.10203834e+00, -1.41362728e+01, -2.92906830e+01,
       -1.61875800e+01, -1.60664785e+01, -1.00659077e+01, -1.57640438e+01,
       -1.72280893e+01, -

In [6]:
x_input = f4_inputs
y_input = f4_outputs

In [7]:
X_max= f4_inputs[np.argmax((f4_outputs))]
print(X_max)

[0.402732 0.392983 0.412051 0.411811]


In [8]:
Y_max= f4_outputs[np.argmax((f4_outputs))]
print(Y_max)

0.48394520320252754


In [9]:
x1 = np.linspace(0.402732-0.05, 0.402732+0.05, 10)
x2 = np.linspace(0.392983-0.05, 0.392983+0.05, 10)
x3 = np.linspace(0.412051-0.05, 0.412051+0.05, 10)
x4 = np.linspace(0.411811-0.05, 0.411811+0.05, 10)
X_grid = []
for i in range(len(x1)):
    for j in range(len(x2)):
        for k in range(len(x3)):
            for l in range(len(x3)):
                X_grid.append([x1[i], x2[j],x3[k],x4[l]])
X_grid = np.array(X_grid)
#0.402732 0.392983 0.412051 0.411811
#0.325901 0.423101 0.406001 0.415501
#0.407765 0.498771 0.295825 0.419007
#0.457765 0.548771 0.345825 0.369007
#0.507765 0.498771 0.395825 0.319007
#[0.557765  , 0.448771  , 0.445825  , 0.269007  ]
#[0.57776561, 0.42877174, 0.42582587, 0.24900741]

In [11]:

real_noise_std = 1e-10 
noise_assumption = 1e-10

rbf_lengthscale = 0.1

beta = 0.09
variable_value = np.linspace(0.001, 0.09, 10)
#beta = np.linspace(0.1, 0.9, 100)
#beta = np.linspace(1, 3, 100)
#beta = np.linspace(1, 50, 100)
result = []
for i in range(len(variable_value)):
    #beta = variable_value[i]
    kernel = RBF(length_scale=rbf_lengthscale, length_scale_bounds='fixed')
    model = GaussianProcessRegressor(kernel = kernel, alpha=noise_assumption)
    model.fit(np.array(x_input),np.array(y_input))
    post_mean, post_std = model.predict(np.array(X_grid), return_std=True)
    post_mean, post_std = post_mean.squeeze(), post_std.squeeze()
    acquisition_function = post_mean + beta* post_std
    result.append([variable_value[i],[X_grid[np.argmax(acquisition_function)]]])

In [12]:
#[0.001, [array([0.55776561, 0.40877174, 0.40582587, 0.26900741]
#[0.001, [array([0.59776561, 0.44877174, 0.40582587, 0.26900741]
#[0.001, [array([0.59776561, 0.44877174, 0.40582587, 0.26900741]
#[0.001, [array([0.55776561, 0.44877174, 0.44582587, 0.26900741])]
#[0.001, [array([0.55776561, 0.44877174, 0.44582587, 0.26900741]
#

In [13]:
result

[[0.001, [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.010888888888888889,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.020777777777777777,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.030666666666666665,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.04055555555555555,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.050444444444444445,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.06033333333333333,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.07022222222222221,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.0801111111111111,
  [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]],
 [0.09, [array([0.39717644, 0.40964967, 0.41760656, 0.39514433])]]]